In [1]:
#!pip install torch torchvision pandas pillow


In [2]:
import pandas as pd

def get_colormap_from_csv(path):
    df = pd.read_csv(path)
    colormap = {}
    for i, row in df.iterrows():
        rgb = (row['r'], row['g'], row['b'])
        colormap[rgb] = i  # index as class label
    return colormap

colormap = get_colormap_from_csv(r"C:\Users\Trijal\Desktop\DL U-net\data\class_dict.csv")


In [3]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np
import torchvision.transforms as transforms

class DeepGlobeDataset(Dataset):
    def __init__(self, root_dir, transform=None, colormap=None):
        self.root_dir = root_dir
        self.filenames = sorted([f for f in os.listdir(root_dir) if f.endswith('_sat.jpg')])
        self.transform = transform
        self.colormap = colormap

    def mask_to_class(self, mask):
        mask = np.array(mask)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int64)
        for rgb, idx in self.colormap.items():
            matches = np.all(mask == rgb, axis=-1)
            label_mask[matches] = idx
        return torch.from_numpy(label_mask).long()

    def __getitem__(self, idx):
        base = self.filenames[idx].replace('_sat.jpg', '')
        img_path = os.path.join(self.root_dir, f"{base}_sat.jpg")
        mask_path = os.path.join(self.root_dir, f"{base}_mask.png")

        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path).convert('RGB')

        # Apply same size to both
        if self.transform:
            image = self.transform(image)
            mask = mask.resize((256, 256), resample=Image.NEAREST)

        mask = self.mask_to_class(mask)  # Now the shape will match model output

        return image, mask


    def __len__(self):
        return len(self.filenames)


In [4]:
import torch.nn as nn

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.enc1 = DoubleConv(3, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)
        self.bottleneck = DoubleConv(512, 1024)

        self.pool = nn.MaxPool2d(2)

        self.up4 = nn.ConvTranspose2d(1024, 512, 2, 2)
        self.dec4 = DoubleConv(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, 2, 2)
        self.dec3 = DoubleConv(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = DoubleConv(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = DoubleConv(128, 64)

        self.out = nn.Conv2d(64, n_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))
        b = self.bottleneck(self.pool(e4))

        d4 = self.dec4(torch.cat([self.up4(b), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))
        return self.out(d1)


In [5]:
from torch.utils.data import DataLoader
import torch.optim as optim

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = DeepGlobeDataset(r"C:\Users\Trijal\Desktop\DL U-net\data\train", transform=transform, colormap=colormap)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(n_classes=len(colormap)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [6]:
from tqdm import tqdm

for epoch in range(5):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)
    
    for images, masks in progress_bar:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}, Total Loss: {total_loss:.4f}")


Epoch 1, Total Loss: 279.1285


Epoch 2, Total Loss: 232.2572


Epoch 3, Total Loss: 212.9073


Epoch 4, Total Loss: 200.3771


Epoch 5, Total Loss: 188.6523


In [17]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [18]:
val_dataset = DeepGlobeDataset(root_dir=r"C:\Users\Trijal\Desktop\DL U-net\data\valid", transform=transform, colormap=colormap)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [19]:
def evaluate_segmentation(model, dataloader, num_classes):
    model.eval()
    iou_scores = []
    dice_scores = []
    total_pixels = 0
    correct_pixels = 0

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            for pred, gt in zip(preds, masks):
                total_pixels += gt.numel()
                correct_pixels += (pred == gt).sum().item()

                for cls in range(num_classes):
                    pred_inds = (pred == cls)
                    gt_inds = (gt == cls)

                    intersection = (pred_inds & gt_inds).sum().item()
                    union = (pred_inds | gt_inds).sum().item()
                    dice = (2 * intersection) / (pred_inds.sum().item() + gt_inds.sum().item() + 1e-6)

                    if union > 0:
                        iou = intersection / (union + 1e-6)
                        iou_scores.append(iou)
                        dice_scores.append(dice)

    pixel_accuracy = correct_pixels / total_pixels
    mean_iou = np.mean(iou_scores)
    mean_dice = np.mean(dice_scores)

    print(f"✅ Evaluation Metrics on Validation Set:")
    print(f"  🔹 Pixel Accuracy      : {pixel_accuracy:.4f}")
    print(f"  🔹 Mean IoU (Jaccard)  : {mean_iou:.4f}")
    print(f"  🔹 Mean Dice (F1 Score): {mean_dice:.4f}")


In [20]:
evaluate_segmentation(model, val_loader, num_classes=len(colormap))


✅ Evaluation Metrics on Validation Set:
  🔹 Pixel Accuracy      : 0.8915
  🔹 Mean IoU (Jaccard)  : 0.3323
  🔹 Mean Dice (F1 Score): 0.3786
